## kf debug

In [123]:
import sagemaker
import boto3

sess = boto3.Session()
account = boto3.client('sts').get_caller_identity().get('Account')
sm   = sess.client('sagemaker')
role = sagemaker.get_execution_role()
# sagemaker_session = sagemaker.Session(boto_session=sess)
sagemaker_session = sagemaker.Session()
bucket = sagemaker.Session().default_bucket()

In [124]:


inference_image = '763104351884.dkr.ecr.ap-northeast-2.amazonaws.com/tensorflow-inference:1.15.2-cpu'
print("inference_image: ", inference_image)
model_data = 's3://sagemaker-ap-northeast-2-189546603447/jobs/TrainingJob-20210408085502-KH4I/output/model.tar.gz'

inference_image:  763104351884.dkr.ecr.ap-northeast-2.amazonaws.com/tensorflow-inference:1.15.2-cpu


In [125]:
from sagemaker.tensorflow.serving import Model

model = Model(model_data=model_data,
              role=role,
              source_dir="code222",              
              entry_point='inference2.py',
              image_uri = inference_image
             ) 


The class sagemaker.tensorflow.serving.Model has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [126]:
import os
os.getcwd()

'/home/ec2-user/SageMaker/kubeflow-pipelines-sagemaker-examples'

In [127]:
import time
endpont_name = 'local-endpoint'

instance_type='local'
deployed_model = model.deploy(
                             endpoint_name= endpont_name,
                             initial_instance_count = 1,
                             instance_type = instance_type,
                             wait=True)


update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Attaching to yxvtnirykg-algo-1-y8yxw
yxvtnirykg-algo-1-y8yxw | INFO:__main__:starting services
yxvtnirykg-algo-1-y8yxw | INFO:__main__:nginx config: 
yxvtnirykg-algo-1-y8yxw | load_module modules/ngx_http_js_module.so;
yxvtnirykg-algo-1-y8yxw | 
yxvtnirykg-algo-1-y8yxw | worker_processes auto;
yxvtnirykg-algo-1-y8yxw | daemon off;
yxvtnirykg-algo-1-y8yxw | pid /tmp/nginx.pid;
yxvtnirykg-algo-1-y8yxw | error_log  /dev/stderr error;
yxvtnirykg-algo-1-y8yxw | 
yxvtnirykg-algo-1-y8yxw | worker_rlimit_nofile 4096;
yxvtnirykg-algo-1-y8yxw | 
yxvtnirykg-algo-1-y8yxw | events {
yxvtnirykg-algo-1-y8yxw |   worker_connections 2048;
yxvtnirykg-algo-1-y8yxw | }
yxvtnirykg-algo-1-y8yxw | 
yxvtnirykg-algo-1-y8yxw | http {
yxvtnirykg-algo-1-y8yxw |   include /etc/nginx/mime.types;
yxvtnirykg-algo-1-y8yxw |   default_type application/json;
yxvtnirykg-algo-1-y8yxw |   access_log /dev/stdout combined;
yxvtnirykg-algo-1-y8yxw |   js_include tensorflow-serving.js;
yxvtnirykg-algo-1-y8yxw | 
yxvtnirykg-alg

In [131]:
file_name = '1000_dog.png'
with open(file_name, 'rb') as f:
    payload = f.read()

import numpy as np
data = np.random.randn(1, 32, 32, 3)
# print("Predicted class is {}".format(np.argmax(predictor.predict(data)['predictions'])))    
deployed_model.predict(data)

{'error': 'Error: 415, Unsupported content type "application/json"'}

yxvtnirykg-algo-1-y8yxw | INFO:python_service:http://gunicorn_upstream/invocations
yxvtnirykg-algo-1-y8yxw | INFO:tfs_utils:sagemaker tfs attributes: 
yxvtnirykg-algo-1-y8yxw | {}
yxvtnirykg-algo-1-y8yxw | ERROR:python_service:exception handling request: Error: 415, Unsupported content type "application/json"
yxvtnirykg-algo-1-y8yxw | Traceback (most recent call last):
yxvtnirykg-algo-1-y8yxw |   File "/sagemaker/python_service.py", line 252, in _handle_invocation_post
yxvtnirykg-algo-1-y8yxw |     res.body, res.content_type = self._handlers(data, context)
yxvtnirykg-algo-1-y8yxw |   File "/sagemaker/python_service.py", line 281, in handler
yxvtnirykg-algo-1-y8yxw |     processed_input = custom_input_handler(data, context)
yxvtnirykg-algo-1-y8yxw |   File "/opt/ml/model/code/inference.py", line 44, in input_handler
yxvtnirykg-algo-1-y8yxw |     _return_error(415, 'Unsupported content type "{}"'.format(context.request_content_type or 'Unknown'))
yxvtnirykg-algo-1-y8yxw |   File "/opt/ml

In [129]:
# endpoint_name = deployed_model.endpoint_name

# # deployed_model.content_type='application/x-image'    
# deployed_model.predict(payload)

In [130]:
import json, boto3
client = boto3.client('runtime.sagemaker')

file_name = '1000_dog.png'
with open(file_name, 'rb') as f:
    payload = f.read()
    
# print("payload: \n", payload)    

response = client.invoke_endpoint(EndpointName= endpoint_name, 
                                  ContentType='application/x-image', 
                                  Body=payload)
print(response['Body'].read())
labels = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

ValidationError: An error occurred (ValidationError) when calling the InvokeEndpoint operation: Endpoint local-endpoint of account 189546603447 not found.

## 엔드포인트 디버깅

In [28]:
from sagemaker.tensorflow.model import TensorFlowModel

training_job_name='TrainingJob-20210408030418-BHHB'
# inference_image = '763104351884.dkr.ecr.ap-northeast-2.amazonaws.com/tensorflow-inference:1.15.2-cpu-py36-ubuntu18.04-v4.4'
# inference_image = '763104351884.dkr.ecr.ap-northeast-2.amazonaws.com/tensorflow-inference:1.15.2-cpu'

cifar_model = TensorFlowModel(
    name='kubeflow-debug',
    source_dir="code",
    entry_point="inference.py",
    # image_uri = inference_image,
    model_data="s3://{}/{}/output/model.tar.gz".format(bucket, training_job_name),
    role=role,
    framework_version="1.15.2",
    sagemaker_session = sagemaker_session
)

## Try to attach job

In [32]:
# from sagemaker.tensorflow.estimator import TensorFlow

# estimator = TensorFlow.attach(training_job_name='TrainingJob-20210408030418-BHHB')
